# Installing Libraries

In [ ]:
# Install all the requirements
!pip install -r https://raw.githubusercontent.com/FranckSoma/ChatGPT_Code_Improver/main/requirements.txt

# Install PIL for image processing
!pip install Pillow


# Dividing Training and Evaluation Images


In [ ]:
def split_data(source_folder, train_folder, eval_folder, eval_ratio):
    """Split data into training and evaluation sets."""
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(eval_folder):
        os.makedirs(eval_folder)

    images = [img for img in os.listdir(source_folder) if img.endswith('.png') or img.endswith('.jpg')]
    random.shuffle(images)

    eval_size = int(len(images) * eval_ratio)

    for img in images[:eval_size]:
        shutil.move(os.path.join(source_folder, img), os.path.join(eval_folder, img))

    for img in images[eval_size:]:
        shutil.move(os.path.join(source_folder, img), os.path.join(train_folder, img))

# Adjust these paths and values
source_folder = 'FFHQ/Images'  # Path to your dataset
train_folder = 'FFHQ/Train'    # Path to the training set folder
eval_folder = 'FFHQ/Eval'      # Path to the evaluation set folder
eval_ratio = 0.2               # Percentage of data to be used for evaluation


split_data(source_folder, train_folder, eval_folder, eval_ratio)


# Training the Model


In [ ]:
python -m torch.distributed.launch --nproc_per_node=2 train_styleswin.py --batch 4 --path /FFHQ/Train --checkpoint_path /trained_model --sample_path /temp --size 256 --G_channel_multiplier 2 --bcr --D_lr 0.0002 --D_sn --ttur --eval_gt_path /FFHQ/Eval --lr_decay --lr_decay_start_steps 775000 --iter 1000000

# PID Score and Demo Images

In [ ]:
python -m torch.distributed.launch --nproc_per_node=1 train_styleswin.py --sample_path /generated_images --size 256 --G_channel_multiplier 2 --ckpt /trained_model --eval --val_num_batches 12500 --val_batch_size 4 --eval_gt_path /FFHQ/Eval
